In [ ]:
!pip install scikit-learn==1.6.1
!pip install opencv-python mediapipe
!pip install lightgbm
!pip install xgboost

In [ ]:
import glob

import cv2
import joblib
import lightgbm
import mediapipe as mp
import numpy as np
import xgboost
import pandas as pd

In [ ]:
loaded_model = joblib.load("stack_model.joblib")
scaler = joblib.load('scaler.pkl')

In [ ]:
def arround_px(x_px, y_px, image, padding_cells):
    total_r = 0
    total_g = 0
    total_b = 0
    padded_space = padding_cells // 2
    total_padded_cells = padding_cells**2
    padded_image = np.pad(
        image,
        pad_width=((padded_space, padded_space), (padded_space, padded_space), (0, 0)),
        mode="constant",
        constant_values=0,
    )
    # paddingする正方形の一片の長さ
    region = padded_image[
        y_px : y_px + padded_space * 2, x_px : x_px + padded_space * 2
    ]
    region_lab = cv2.cvtColor(region, cv2.COLOR_BGR2Lab)
    L, a, b = cv2.split(region_lab)
    # Lを正規化（0〜255）
    L_norm = cv2.normalize(L, None, 0, 255, cv2.NORM_MINMAX)
    # 元のa, bとマージしてBGRに戻す
    region_norm_lab = cv2.merge([L_norm, a, b])
    region_mean_lab = region_norm_lab.mean(axis=(0, 1))

    return region_mean_lab


def check_make(image, landmark, landmark_id, height, width, padding_cells):
    lm = face_landmarks.landmark[landmark_id]
    x_px = min(int(lm.x * width), width - 1)
    y_px = min(int(lm.y * height), height - 1)
    l, a, b = arround_px(x_px, y_px, image, padding_cells)
    # 指定したランドマークの周辺が措置を取得し、画素値の平均を取る
    df = pd.DataFrame({
    "L": [l],
    "a": [a],
    "b": [b]
})
    df[["L", "a", "b"]] = scaler.transform(df[["L", "a", "b"]])
    l, a, b = df.loc[0]
    X = [l, a, b]
    print(X)
    onehot_label = one_hot_landmark(landmark_id)
    X = np.concatenate((X, onehot_label))
    X = X.reshape(1, -1)
    pre_score = loaded_model.predict(X)
    print(X)

    print(f"pre_score: {pre_score}")
    return pre_score[0]


def one_hot_landmark(landmark_id):
    # 対象となる landmark_id のリスト
    landmark_ids = [164, 165, 167, 391, 393, 322, 92]

    # landmark_id がリストに含まれているか確認
    if landmark_id not in landmark_ids:
        raise ValueError(
            f"指定された landmark_id {landmark_id} は対象のリストに含まれていません。"
        )

    # ワンホットエンコーディングの配列を初期化
    one_hot = np.zeros(len(landmark_ids), dtype=int)

    # 対応するインデックスに 1 を設定
    index = landmark_ids.index(landmark_id)
    one_hot[index] = 1

    return one_hot


# MediaPipe の初期化
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 顔のランドマーク描画用スタイル
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
# Webカメラを起動
# 0の時たまにスマホのカメラが起動するので１
cap = cv2.VideoCapture(1)
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 150)
color = (255, 255, 255)
font_scale = 2
thickness = 3
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("カメラからフレームを取得できませんでした。")
            break

        # パフォーマンス向上のため画像を一時的に「書き込み不可」に
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # 画像を「書き込み可能」に戻す
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        hige_ids = [164, 165, 167, 391, 393, 322, 92]
        # ランドマークの描画
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                pass;
                # mp_drawing.draw_landmarks(
                #     image=image,
                #     landmark_list=face_landmarks,
                #     connections=mp_face_mesh.FACEMESH_TESSELATION,
                #     landmark_drawing_spec=None,
                #     connection_drawing_spec=drawing_spec)
            # 顔のnodeの横にランドマークidを描画
            h, w, _ = image.shape
            # 顔から青髭などがあるかを確認する際に指定したピクセルの周辺n*nマスの画素値の平均を用いているがその際に用いるn*nマスの正方形の直径(px)
            left_lm = face_landmarks.landmark[234]
            right_lm = face_landmarks.landmark[454]
            left_x = int(left_lm.x * w)
            right_x = int(right_lm.x * w)
            # 顔の幅
            face_width_px = abs(right_x - left_x)
            # 顔の位置に基づいて動的にpadding_cellsを決定（例えば顔幅の1/5など）
            padding_cells = max(11, int(face_width_px * 0.1))  # 最小値を保証
            ##お好みのランドマークのインデックスを指定するとその値のrgb値を取得
            hige_ids = [164, 165, 167, 391, 393, 322, 92]
            for lm_id in hige_ids:
                has_beard = check_make(
                    image, face_landmarks, lm_id, h, w, padding_cells
                )
                if has_beard:
                    cv2.putText(
                        image,
                        "HIGE",
                        org,
                        font,
                        font_scale,
                        color,
                        thickness,
                        cv2.LINE_AA,
                    )
                else:
                    cv2.putText(
                        image,
                        "NOT HIGE",
                        org,
                        font,
                        font_scale,
                        color,
                        thickness,
                        cv2.LINE_AA,
                    )

        # 表示

        cv2.imshow("MediaPipe FaceMesh", image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()